### ***Battle of Neighborhoods (Part II)***

***Project Name:***"Perfect place in NYC to put a new music school"

***Background:*** 
There are a few well known music schools in NYC, Manhattan and all cater to different audiences. We will look to open a school in another borough

***Goal:***
This project will be finding the ***best*** place to start a music school in a popular place in an NYC borough

1)The project will be centered around the borough that yields the best results , not necessarily meaning the music school ***has*** to be in a specific borough, but the starting point to find nearby music schools will start at the chosen borough that is yielded by results of nearby API unless results specify differently

***Data Description***

1a) The first step is to load the data for New York City and pick a borough to work with

2) Using FourSquare location data, a visual map will show a map of New York in general 

3) Then we will find the latitude and longitude for our chosen borough (Queens)

4) Find out which city in Queens needs a music school

5) How many music schools are near our chosen latitude and longitude coordinates?

6) How many are in our city?

7) Reflecting on the competition, we can see if our city is good for a new music school

***Methodology***

1) Import Libraries to prep the data

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


***2) Download General New York data***

In [2]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [63]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [5]:
neighborhoods_data = newyork_data['features']

In [6]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

***Change data to dataframe format***

In [7]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


***Now we have figured out which boroughs to look at and how many neighborhoods to sort through!***

***Next: Latitude and Longitude for NYC***

In [10]:
address = 'New York City, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of New York City are 40.7308619, -73.9871558.


In [11]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

***Important Step: now, different from before, we will be searching for data specifically in the borough of Queens***

In [12]:
queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
queens_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


***Now: We found the geographical coordinates of the borough of Queens, for our school***

In [13]:
address = 'Queens, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Queens are 40.6524927, -73.7914214158161.


***Now we take a look at the neighborhoods within Queens***

In [14]:

map_queens = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(queens_data['Latitude'], queens_data['Longitude'],queens_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_queens)  
    
map_queens

***Initiate Foursquare Credentials for further data retrieval***

In [15]:
CLIENT_ID = '5KMYNQOVUJDHMMPZGK1G3AYV55LOTBEFMHE4LOUJ1J2CB3CP'
CLIENT_SECRET = 'I0O1CFSJCLQSNLASS5T4SK1PQ2ZOHDEW341TIRYDUXDLDO4V'
VERSION = '20181106'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5KMYNQOVUJDHMMPZGK1G3AYV55LOTBEFMHE4LOUJ1J2CB3CP
CLIENT_SECRET:I0O1CFSJCLQSNLASS5T4SK1PQ2ZOHDEW341TIRYDUXDLDO4V


In [16]:
queens_data.loc[0, 'Neighborhood']

'Astoria'

***We have chosen Astoria as our starting point***

In [18]:
neighborhood_latitude = queens_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = queens_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = queens_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Astoria are 40.76850859335492, -73.91565374304234.


***The established Longitude and Latitude Values for Astoria are:***

40.76850859335492, -73.91565374304234

***FourSquare Implimentation***

***Now we use FourSquare to find the best place!***

In [24]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 
 

'https://api.foursquare.com/v2/venues/explore?&client_id=5KMYNQOVUJDHMMPZGK1G3AYV55LOTBEFMHE4LOUJ1J2CB3CP&client_secret=I0O1CFSJCLQSNLASS5T4SK1PQ2ZOHDEW341TIRYDUXDLDO4V&v=20181106&ll=40.76850859335492,-73.91565374304234&radius=500&limit=100'

In [25]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5be25bf96a6071168e338293'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Astoria',
  'headerFullLocation': 'Astoria, Queens',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 108,
  'suggestedBounds': {'ne': {'lat': 40.773008597854925,
    'lng': -73.90972309237958},
   'sw': {'lat': 40.76400858885492, 'lng': -73.9215843937051}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bdf502a89ca76b062b75d5e',
       'name': 'Favela Grill',
       'location': {'address': '33-18 28th Ave',
        'crossStreet': 'btwn 33rd & 34th St.',
        'lat': 40.76734843380796,
        'lng': -73.9178974

***From viewing the data for Astoria Queens, we can see there may be a demand for music schools!***

In [127]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [128]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Favela Grill,Brazilian Restaurant,40.767348,-73.917897
1,Orange Blossom,Gourmet Shop,40.769856,-73.917012
2,Titan Foods Inc.,Gourmet Shop,40.769198,-73.919253
3,Off The Hook,Seafood Restaurant,40.767200,-73.918104
4,CrossFit Queens,Gym,40.769404,-73.918977


***Since the above only shows general venues | we instead change from explore--> to search for music schools***

***Now we change our api address to show us music schools in Astoria Long/Lat***

In [93]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
    
url = 'https://api.foursquare.com/v2/venues/search?client_id=5KMYNQOVUJDHMMPZGK1G3AYV55LOTBEFMHE4LOUJ1J2CB3CP&client_secret=I0O1CFSJCLQSNLASS5T4SK1PQ2ZOHDEW341TIRYDUXDLDO4V&v=20181106&ll=40.76850859335492,-73.91565374304234&query=music school'
CLIENT_ID, 
CLIENT_SECRET, 
VERSION, 
latitude, 
longitude,
url 

'https://api.foursquare.com/v2/venues/search?client_id=5KMYNQOVUJDHMMPZGK1G3AYV55LOTBEFMHE4LOUJ1J2CB3CP&client_secret=I0O1CFSJCLQSNLASS5T4SK1PQ2ZOHDEW341TIRYDUXDLDO4V&v=20181106&ll=40.76850859335492,-73.91565374304234&query=music school'

***"Res2" represents each school that we near Astoria with updated NPI***

In [94]:
res2 = requests.get(url).json()
res2

{'meta': {'code': 200, 'requestId': '5be269bb351e3d404a8f061d'},
 'response': {'venues': [{'id': '4af5c8b5f964a520d0fc21e3',
    'name': 'Brooklyn Music School',
    'location': {'address': '126 Saint Felix St',
     'lat': 40.68611617889807,
     'lng': -73.97728336212968,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.68611617889807,
       'lng': -73.97728336212968}],
     'distance': 10542,
     'postalCode': '11217',
     'cc': 'US',
     'city': 'Brooklyn',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['126 Saint Felix St',
      'Brooklyn, NY 11217',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d13b941735',
      'name': 'School',
      'pluralName': 'Schools',
      'shortName': 'School',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1541564859',
    'hasPerk': False},
   {'id': '4b6ca949f964a520374a2c

***Visualize Music Schools in the area via a dataframe***

In [124]:
dfm1 = pd.read_csv('MSC2.csv')
dfm1

,ID,Venues_Name,Venues__location__city,Venues__location__formattedAddress__002,Venues__location__address,Venues__location__crossStreet,Venues__location__lat,response__venues__location__lng,response__venues__location__labeledLatLngs__label,response__venues__location__labeledLatLngs__lat,Venues__location__labeledLatLngs__lng,Venues__location__distance,Venues__location__postalCode,Venues__location__cc,Venues__location__state,Venues__location__country,Venues__location__formattedAddress__001,response__venues__location__formattedAddress__003,response__venues__categories__id,response__venues__categories__name,response__venues__categories__pluralName,response__venues__categories__shortName,response__venues__categories__icon__prefix,response__venues__categories__icon__suffix,response__venues__categories__primary,response__venues__venuePage__id,response__venues__referralId,response__venues__hasPerk
0,4af5c8b5f964a520d0fc21e3,Brooklyn Music School,Brooklyn,"Brooklyn, NY 11217",126 Saint Felix St,NaN,40.686116,-73.977283,display,40.686116,-73.977283,10542,11217.0,US,NY,United States,126 Saint Felix St,United States,4bf58dd8d48988d13b941735,School,Schools,School,https://ss3.4sqi.net/img/categories_v2/buildin...,.png,True,NaN,v-1541565458,False
1,4b6ca949f964a520374a2ce3,Third Street Music School,New York,"New York, NY 10003",235 E 11th St,btwn 2nd & 3rd,40.730866,-73.986936,display,40.730866,-73.986936,7327,10003.0,US,NY,United States,235 E 11th St (btwn 2nd & 3rd),United States,4f04b10d2fb6e1c99f3db0be,Music School,Music Schools,Music School,https://ss3.4sqi.net/img/categories_v2/buildin...,.png,True,NaN,v-1541565458,False
2,4b5a20c0f964a520f7ae28e3,Hoff-Barthelson Music School,Scarsdale,"Scarsdale, NY 10583",25 School Ln,NaN,40.987674,-73.804208,display,40.987674,-73.804208,26138,10583.0,US,NY,United States,25 School Ln,United States,4bf58dd8d48988d1f2931735,Performing Arts Venue,Performing Arts Venues,Performing Arts,https://ss3.4sqi.net/img/categories_v2/arts_en...,.png,True,NaN,v-1541565458,False
3,4f5c0013e4b0c83983186872,Turtle Bay Music School,New York,"New York, NY 10022",244 E 52nd St,2nd Avenue,40.756377,-73.968166,display,40.756377,-73.968166,4628,10022.0,US,NY,United States,244 E 52nd St (2nd Avenue),United States,50328a8e91d4c4b30a586d6c,Non-Profit,Non-Profits,Non-Profit,https://ss3.4sqi.net/img/categories_v2/buildin...,.png,True,49218097.0,v-1541565458,False
4,4ab2babcf964a5200b6c20e3,Greenwich House Music School,New York,"New York, NY 10014",Barrow st,7th Ave,40.732122,-74.004388,display,40.732122,-74.004388,8508,10014.0,US,NY,United States,Barrow st (7th Ave),United States,4bf58dd8d48988d1e5931735,Music Venue,Music Venues,Music Venue,https://ss3.4sqi.net/img/categories_v2/arts_en...,.png,True,64944385.0,v-1541565458,False
5,52b3a96c498e5f3a9b735944,Shobo Music School!,Cliffside Park,United States,NaN,NaN,40.829066,-73.980251,display,40.829066,-73.980251,8664,NaN,US,NJ,United States,"Cliffside Park, NJ",NaN,4f04b10d2fb6e1c99f3db0be,Music School,Music Schools,Music School,https://ss3.4sqi.net/img/categories_v2/buildin...,.png,True,NaN,v-1541565458,False
6,4d88dcadb799a35dfe828a63,Music School Inc,Mineola,"Mineola, NY 11501",71 Jericho Tpke,NaN,40.750222,-73.637538,display,40.750222,-73.637538,23538,11501.0,US,NY,United States,71 Jericho Tpke,United States,4bf58dd8d48988d174941735,Coworking Space,Coworking Spaces,Coworking Space,https://ss3.4sqi.net/img/categories_v2/buildin...,.png,True,NaN,v-1541565458,False
7,4c910c0aae96a0931d34a046,Music School,New York,"New York, NY 10025",280 Riverside Dr,100,40.797958,-73.972960,display,40.797958,-73.972960,5837,10025.0,US,NY,United States,280 Riverside Dr (100),United States,4bf58dd8d48988d1e5931735,Music Venue,Music Venues,Music Venue,https://ss3.4sqi.net/img/categories_v2/arts_en...,.png,True,NaN,v-1541565458,False
8,4dc5a2c0c65b89d3ca743a55,Music School at Lighthouse International,New York,"New York, NY 10022",111 E 59th St,Lexington Ave.,40.763461,-73.968781,display,40.763461,-73

In [125]:
dfm1.shape

(30, 28)

***Now we segment the music schools in Queens only***

In [126]:
dfm2 = pd.read_csv('QueensMSC - Sheet1.csv')
dfm2

,ID,Venues_Name,Venues__location__city,Venues__location__formattedAddress__002,Venues__location__address,Venues__location__crossStreet,Venues__location__lat,response__venues__location__lng,response__venues__location__labeledLatLngs__label,response__venues__location__labeledLatLngs__lat,Venues__location__labeledLatLngs__lng,Venues__location__distance,Venues__location__postalCode,Venues__location__cc,Venues__location__state,Venues__location__country,Venues__location__formattedAddress__001,response__venues__location__formattedAddress__003,response__venues__categories__id,response__venues__categories__name,response__venues__categories__pluralName,response__venues__categories__shortName,response__venues__categories__icon__prefix,response__venues__categories__icon__suffix,response__venues__categories__primary,response__venues__venuePage__id,response__venues__referralId,response__venues__hasPerk
0,4d3dfadf6e0aa1cda694f02c,Song of Songs Music School,Flushing,"Flushing, NY 11358",171-74 46th Ave,NaN,40.753403,-73.793708,display,40.753403,-73.793708,10418,11358.0,US,NY,United States,171-74 46th Ave,United States,4f04b10d2fb6e1c99f3db0be,Music School,Music Schools,Music School,https://ss3.4sqi.net/img/categories_v2/buildin...,.png,True,59747545.0,v-1541565458,False
1,4d12889a12916dcbcab3d68a,East-West Music School,Flushing,"Flushing, NY 11358",161-10 Northern Blvd,NaN,40.762540,-73.804509,display,40.762540,-73.804509,9394,11358.0,US,NY,United States,161-10 Northern Blvd,United States,4f04b10d2fb6e1c99f3db0be,Music School,Music Schools,Music School,https://ss3.4sqi.net/img/categories_v2/buildin...,.png,True,55428526.0,v-1541565458,False
2,52504740498e7219a2c08f06,Amadeus Music School,Elmhurst,"Elmhurst, NY 11373",90-36 Corona Ave,91st Place,40.742286,-73.873994,display,40.742286,-73.873994,4567,11373.0,US,NY,United States,90-36 Corona Ave (91st Place),United States,4f04b10d2fb6e1c99f3db0be,Music School,Music Schools,Music School,https://ss3.4sqi.net/img/categories_v2/buildin...,.png,True,NaN,v-1541565458,False
3,4cf289c36195721ea00348c1,Amadeus music school,Flushing,"Flushing, NY",Main street and northern blvd,NaN,40.762615,-73.833380,display,40.762615,-73.833380,6967,NaN,US,NY,United States,Main street and northern blvd,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,v-1541565458,False
4,5bb2c20e35f9830039b9dac1,Further Music School (Forest Hills),Forest Hills,"Forest Hills, NY 11375",110-06 72nd Ave Ste MD2,NaN,40.721079,-73.840796,display,40.721079,-73.840796,8230,11375.0,US,NY,United States,110-06 72nd Ave Ste MD2,United States,4f04b10d2fb6e1c99f3db0be,Music School,Music Schools,Music School,https://ss3.4sqi.net/img/categories_v2/buildin...,.png,True,NaN,v-1541565458,False
5,4d3c4505d2c8f04ddf1c6072,Juliet Music School,Forest Hills,"Forest Hills, NY 11375",96-09 69th Ave,NaN,40.717088,-73.852114,display,40.717088,-73.852114,7841,11375.0,US,NY,United States,96-09 69th Ave,United States,4bf58dd8d48988d1f2931735,Performing Arts Venue,Performing Arts Venues,Performing Arts,https://ss3.4sqi.net/img/categories_v2/arts_en...,.png,True,NaN,v-1541565458,False
6,4f3fca4be4b02a189680e476,Kts Music School,Ozone Park,United States,NaN,NaN,40.679373,-73.844607,display,40.679373,-73.844607,11592,11417.0,US,NY,United States,"Ozone Park, NY 11417",NaN,4bf58dd8d48988d1ab941735,Student Center,Student Centers,Student Center,https://ss3.4sqi.net/img/categories_v2/educati...,.png,True,NaN,v-1541565458,False


### ***Results***

***As we can see from the dataframe, there are only 7 music schools in Queens***

***Additionally, there are no music schools present in Astoria, therefore concluding from data-  Astoria would be an idea place to start a new music school!***

### ***Discussion***

***observations***
 I manually modified my API to find music schools and didn't realize the syntax was so simple!

2)Also did not realize there was such a shortage of music schools within the borough of Queens, specifically a lack in Astoria.

***recommendations***
In my data retrieval I noticed that Long island and Flushing are inundated with schools that are not necessarily listed on these dataframes, so definitely the ideal place would be long island city or Astoria for a fresh start


### ***Conclusion***

From this project, I conclude that Manhattan is the best place to enroll in music schools due to the variety
However, to open a new school business would be best in Queens or Brooklyn.


### ***End***